# Tic Tac Toe

In [1]:
import os
import pickle
import tqdm

In [2]:
from ttt5_funcs import *

2022-11-18 12:42:00.363290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import tqdm

see page https://medium.com/swlh/tic-tac-toe-and-deep-neural-networks-ea600bc53f51

In [3]:
history = simulateGame()
board = movesToBoard(history)
printBoard(board)

|X X O   X |
|  X X O O |
|X X X X O |
|  O X O X |
|O O O O O |
Winner: 2


In [4]:
games = [simulateGame() for _ in tqdm.tqdm(range(10000))]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:20<00:00, 486.18it/s]


In [21]:
gameStats(games)
print()
gameStats(games, player=2)

Results for player 1:
Wins: 2473 (24.7%)
Loss: 1566 (15.7%)
Draw: 5961 (59.6%)

Results for player 2:
Wins: 1566 (15.7%)
Loss: 2473 (24.7%)
Draw: 5961 (59.6%)


In [27]:
np.array(np.nonzero(movesToBoard(games[1])))

array([[0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4],
       [0, 1, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4]])

## DNN

In [ ]:
# model = getModel()
# X_train, X_test, y_train, y_test = gamesToWinLossData(games)
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100)

# model.save_weights("./results/ttt5")

In [5]:
model = getModel()
model.load_weights("./results/ttt5")

2022-11-18 07:14:02.503612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import time

## Simulations

In [6]:
games2 = []

In [7]:
start = time.perf_counter()
games2 += [simulateGame(p1=model) for _ in tqdm.tqdm(range(100))]
print(time.perf_counter()-start)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:34<00:00,  5.15s/it]

514.849219706


In [8]:
print("[games2]=", len(games2))
gameStats(games2)
print()
gameStats(games2, player=2)

[games2]= 100
Results for player 1:
Wins: 20 (20.0%)
Loss: 23 (23.0%)
Draw: 57 (57.0%)

Results for player 2:
Wins: 23 (23.0%)
Loss: 20 (20.0%)
Draw: 57 (57.0%)


In [4]:
def gen_tournament(p1=None, p2=None, n=100, rnd=0.9):
    t_games = []
    n_fault = 0
    for _ in tqdm.tqdm(range(n)):
        try:
            moves = simulateGame(p1=p1, p2=p2, rnd=rnd)
            winner = getWinner(movesToBoard(moves))
            t_games = t_games + [(winner, moves)]
        except:
            n_fault = n_fault + 1
    return t_games, n_fault

In [18]:
epoch = 4

dir_name = "./results/epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)

#
old_model = getModel()
old_model.load_weights("./results/epoch_"+str(epoch-1)+"/model");
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.9)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
#
print("Training new model")
X_train, X_test, y_train, y_test = gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")

Simulation


  0%|                                                                                                                                                                                   | 0/100 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [16:30<00:00,  9.90s/it]


Training new model


## Tournament between different epochs

In [26]:
model1 = getModel()
model1.load_weights("./results/epoch_2/model")

In [37]:
model1 = getModel()
with open("./results/epoch_0/t_games.dat","rb") as file:
    tg = pickle.load(file)
X_train, X_test, y_train, y_test = gamesToWinLossData([g[1] for g in tg])
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)

In [7]:
tg, nf  = gen_tournament(p1=None, p2=None, n=20)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 344.05it/s]


In [11]:
winners = np.array([g[0] for g in tg])

In [13]:
np.sum(winners == 0)

13

In [18]:
n_models = 5
TResults = [[0 for e1 in range(n_models)] for e2 in range(n_models)]
for e1 in range(n_models):
    model1 = getModel()
    model1.load_weights("./results/epoch_"+str(e1)+"/model").expect_partial()
    for e2 in range(e1, n_models):
        print("Play between ", e1, " and ", e2)
        model2 = getModel()
        model1.load_weights("./results/epoch_"+str(e2)+"/model").expect_partial()
        tg, n_faults = gen_tournament(p1=model1, p2=model2, n=1, rnd=0)
        TResults[e1][e2]=tg
        winners = np.array([g[0] for g in tg])
        nDraws = np.sum(winners == 0)
        nW1 = np.sum(winners == 1)
        nW2 = np.sum(winners == 2)
        print("(nD, nW1, nW2)=", (nDraws, nW1, nW2), " n_faults=", n_faults)
        

Play between  0  and  0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.01s/it]


(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  0  and  1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.92s/it]


(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  0  and  2


...

(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  3  and  4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.25s/it]


(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  4  and  4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.96s/it]

(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0


In [52]:
from IPython.display import display, clear_output

In [61]:
moves = TResults[0][4][0][1]
for i in range(len(moves)+1):
    clear_output()
    print("Move ", i)
    display(printBoard(movesToBoard(moves[:i])))
    input()


Move  25
|X O X X X |
|X X O O O |
|O O O X O |
|O X O X O |
|O X X X X |
Winner: 0


None

## Interactive

In [ ]:
def showBoard(board, cols = {1:"r", 2:"g"}, d=0.1):
    plt.xlim(0, len(board))
    plt.ylim(0, len(board[0]))
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j]>0:
                plt.fill(i+np.array([d,d,1-d,1-d]),j+np.array([d,1-d,1-d, d]), cols[ board[i][j]])
    plt.show()
    print("Winner: ", getWinner(board))

In [ ]:
def make_move(board, p):
    showBoard(board)
    coord = plt.ginput(1)[0]
    plt.close()
    i = int(np.floor(coord[0]))
    j = int(np.floor(coord[1]))
    board[i][j]=p

### computer first

In [ ]:
board = initBoard()
showBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

### Human first

In [ ]:
board = initBoard()
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
showBoard(board)

In [ ]:
%matplotlib qt
make_move(board,2)
%matplotlib inline
showBoard(board)

### Ramdom vs Model

In [ ]:
games3= []
for n in range(10):
    start = time.perf_counter()
    games3 += [simulateGame(p2=model) for _ in range(100)]
    print("n=", n, "time=", (time.perf_counter()-start)/60, " mins")

In [ ]:
print("[games3]=", len(games4))
gameStats(games3)
print()
gameStats(games3, player=2)

### Model vs Human

In [ ]:
board = initBoard()
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[0][0] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[1][2] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[2][0] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[0][1] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

### Model vs Human, rnd = 2

In [ ]:
board = initBoard()
move = bestMove(board, model, 1, rnd=2)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[1][1] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 1, rnd=2)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[0][2] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 1, rnd=2)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[2][1] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 2)
board[move[0]][move[1]] = 1
printBoard(board)

In [ ]:
board[0][0] = 2
printBoard(board)

In [ ]:
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

### Human vs Model

In [ ]:
board = initBoard()
board[0][0]=1
printBoard(board)

In [ ]:
move = bestMove(board, model, 2)
board[move[0]][move[1]] = 2
printBoard(board)

In [ ]:
board[1][0] = 1
printBoard(board)

In [ ]:
move = bestMove(board, model, 2)
board[move[0]][move[1]] = 2
printBoard(board)

In [ ]:
board[0][2] = 1
printBoard(board)

In [ ]:
move = bestMove(board, model, 2)
board[move[0]][move[1]] = 2
printBoard(board)

In [ ]:
board[0][1] = 1
printBoard(board)

### Model vs Model

In [ ]:
games4= []

In [ ]:
for n in range(10):
    start = time.perf_counter()
    games4 += [simulateGame(p1=model, p2=model) for _ in range(100)]
    print("n=", n, "time=", (time.perf_counter()-start)/60, " mins")

In [ ]:
print("[games4]=", len(games4))
gameStats(games4)
print()
gameStats(games4, player=4)

In [ ]:
printBoard(movesToBoard(games2[20]))